# Tensorflow 실습 : TFRecords

In [1]:
import os
import tensorflow as tf
import numpy as np
from glob import glob

## TFRecords 파일을 이용하기
- 일반적으로 데이터 양이 많지 않을 때는 모든 데이터를 load한 후에, `tf.data.Dataset.from_tensor_slices`를 이용하면 dataset으로 만들 수 있음
- TFRecords 활용이 필요한 경우
  - **데이터 양이 매우 많아서 한번에 load하면 메모리 용량을 초과할 때**
- 일반적으로 데이터를 input으로 사용하기 위해 필요한 절차 (Extract, Transform, Load)
  - Extract: 파일로 저장된 데이터를 불러옴 (csv file, numpy file, tfrecord file 등)
  - Transform: 원하는 방식으로 전처리하거나, augmentation을 적용해야 하고(이미지 데이터 rotation, 텍스트 데이터 벡터화, 오디오 데이터의 signal process 등), random하게 batch를 구성
  - Load: transformed data를 GPU/TPU (accelerator devices)에 올려서 연산

- TFRecords 형식은 Extract 과정에서 효율적으로 데이터를 불러오기 위한 저장 방식
- 많은 양의 데이터를 한번에 모두 불러오는 것은 **memory 사용량에서 비효율적이거나 불가능함**
- 데이터를 여러 개의 파일로 나누어 저장하고, 원하는 시점에 필요로하는 데이터를 불러오는 방식으로 문제를 해결
- scalar 값을 가지는 데이터를 string 형태로 변환하여 저장하고, 불러와서 원래의 값을 복원하여 사용


### tf.Example
- tfrecord file로 저장하기 위해, tensorflow에서는 `tf.train.Example`이라는 형식을 지원함
- `tf.Example`은  {"string": tf.train.Feature}의 형태를 가짐
- 아래 3개의 함수는, 특정 data type으로 이루어진 **list**를 input으로 받아서, tf.Example에 알맞은 tf.train.Feature 형태로 변환하는 역할을 함
- `_bytes_feature`
  - `string`
  - `byte`
- `_float_feature`
  - `float (float32)`
  - `double (float64)`
- `_int64_feature`
  - `bool`
  - `enum`
  - `int32`
  - `uint32`
  - `int64`
  - `uint64`

In [2]:
# 다음 함수를 사용하여 값을 tf.Example과 호환되는 유형으로 변환 할 수 있습니다.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

### tfrecord 파일로 저장하기


- 여기서는 가상의 데이터 (x, y)를 총 3개 생성하고, 각각을 파일로 저장

In [3]:
x_shape = [2, 4]
for i in range(3):
    filename = '%d.tfrecord' % i

    # TFRecordWriter를 이용(저장 경로 설정)
    writer = tf.io.TFRecordWriter(filename)

    # 가상의 데이터 x,y
    # x는 (2, 4)의 float matrix, y는 integer
    x = np.random.normal(size=x_shape)
    print('x : ', x)
    y = np.random.randint(0, 10, 1)
    print('y : ', y, '\n')

    # x는 float, y는 integer이므로 알맞은 함수를 이용하여 변환 후, data라는 dictionary로 표현
    # x.flatten() : x는 (2, 4)의 matrix이므로, float 값으로 이루어진 하나의 list가 아님 => flatten을 이용하여 하나의 list로 변환
    data = dict()
    data['x'] = _float_feature(x.flatten())
    data['y'] = _int64_feature(y)

    # data dictionary를 이용하여, tf.train.Example 형태로 변환
    data = tf.train.Example(features=tf.train.Features(feature=data))

    # SerializeToString : tf.train.Example의 데이터를 string의 형태로 변환
    # writer를 이용하여, 데이터를 파일로 저장하고 writer를 close
    writer.write(data.SerializeToString())
    writer.close()

x :  [[-0.65522189  1.46774334 -1.19310953  0.41142919]
 [-0.76515952  1.32940409 -0.85855623 -0.11702233]]
y :  [8] 

x :  [[-0.83695887 -1.22300236 -0.63874524 -0.25347132]
 [ 1.81263417  0.34792715  1.39463733  0.93803662]]
y :  [1] 

x :  [[0.99753871 0.44600537 2.15075709 0.20510792]
 [1.96715891 0.32951457 0.86613625 0.48067348]]
y :  [3] 



### 저장된 파일 확인하기


In [4]:
# 저장된 tfrecord 파일의 path를 list로 만들기
filenames = glob('*.tfrecord')

# 파일 이름으로 정렬
filenames = sorted(filenames)
print(filenames)

# TFRecordDataset : tfrecord 파일 list를 넣어주면, 해당 파일을 load 해주는 Dataset
# 여기서는 저장된 3개의 tfrecord file을 dataset 객체로 변환
dataset = tf.data.TFRecordDataset(filenames)

# 아래는 불러온 3개의 data를 출력한 결과
# string 형태로 저장되어 있는 파일을 불러온 것으로, 원래의 x, y로 변환이 필요함
for i in dataset:
    print(i)

['0.tfrecord', '1.tfrecord', '2.tfrecord']
tf.Tensor(b'\n7\n)\n\x01x\x12$\x12"\n \x9f\xbc\'\xbf\x03\xdf\xbb?\xd0\xb7\x98\xbf\xd9\xa6\xd2>\x7f\xe1C\xbf\xea)\xaa?W\xca[\xbfh\xa9\xef\xbd\n\n\n\x01y\x12\x05\x1a\x03\n\x01\x08', shape=(), dtype=string)
tf.Tensor(b'\n7\n)\n\x01x\x12$\x12"\n \xf0BV\xbfW\x8b\x9c\xbf\xcf\x84#\xbf\xfe\xc6\x81\xbee\x04\xe8?\x82#\xb2>z\x83\xb2?+#p?\n\n\n\x01y\x12\x05\x1a\x03\n\x01\x01', shape=(), dtype=string)
tf.Tensor(b'\n7\n)\n\x01x\x12$\x12"\n \xb2^\x7f?\xd1Z\xe4>\x01\xa6\t@\xd0\x07R>\xdd\xcb\xfb?"\xb6\xa8>\x1b\xbb]?\xd6\x1a\xf6>\n\n\n\x01y\x12\x05\x1a\x03\n\x01\x03', shape=(), dtype=string)


- 각각의 파일은 dataset에 대해 for문을 돌면서, load가 필요한 시점에 파일이 load됨
  - **따라서 모든 데이터를 한번에 메모리에 올리지 않음**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### string에서 원래 데이터 형태로 복원하기

- string 형태의 데이터를 원래 형태로 변환하기 위해, `parse_record`를 먼저 정의

In [ ]:
# string 형태의 데이터를 원래 형태로 변환하는 함수
def parse_record(example_proto, x_shape):
    length = x_shape[0] * x_shape[1]

    # 저장할때 사용한 dictionary의 형태를 동일하게 표현
    feature_description = {
        # x는 length(2 * 4)의 길이를 가지고 tf.float32 type
        # 길이를 정확하게 지정하지 않으면 오류 발생
        'x': tf.io.FixedLenFeature([length], tf.float32),
        
        # y는 tf.int64 type. 1개의 값이므로 길이는 생략
        'y': tf.io.FixedLenFeature([], tf.int64)
    }

    # parsed: 각각의 x,y를 parsing한 결과는 dictionary 형태로 표현됨
    parsed = tf.io.parse_single_example(example_proto, feature_description)

    x = parsed['x']
    # x는 key값을 넣어 불러온 후, 원래 shape에 맞게 reshape해주는 과정이 반드시 필요함
    x = tf.reshape(x, x_shape)
    y = parsed['y']
    return x, y


- `parse_record` 함수를 map function을 통해 dataset의 모든 데이터에 대해 적용

In [ ]:
parsed_dataset = dataset.map(lambda x: parse_record(x, x_shape))

In [ ]:
# 총 3개의 데이터를 batch size 2를 적용하여 load
for x, y in parsed_dataset.batch(2):
    print('x : ', x)
    print('y : ', y, '\n')

- 이처럼 string 상태의 tfrecord file의 값을 parsing한 후에는, dataset에서 제공하는 batch, shuffle, prefetch 등을 자유롭게 활용하여 data pipeline을 완성할 수 있음
